In [43]:
import glob

genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = 27  # Which month and days result you want to visualize in table
month = 10
repetitionValue = 15
curSavePath = f"{genSavePath}{day}-{month}-{repetitionValue}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = [2]
onlySignificant = True
minAcc = 0.3

In [44]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]

                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])

                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()


# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
table = pd.DataFrame(np.array(subjectResultArray), columns=headers)
display(table)


c:\Users\Luna\anaconda3\envs\mentalab3\lib\site-packages\ipykernel_launcher.py:170: RuntimeWarning: invalid value encountered in true_divide


(1, 396)
(10, 396)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.325,None,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,None,None,0.3,0.325,0.325,0.325,None,None,None,None,None,None,None,0.35,0.325,0.325,0.325,0.325,0.375,0.3,0.375,0.375,None,0.3,None,0.325,0.325,None,None,None,None,None,None,None,None,None,None,None,None,0.325,0.3,0.325,0.325,0.325,None,None,None,None,None,None,None,0.3,0.35,0.35,0.325,None,None,None,None,None,None,None,None,0.4,0.375,0.4,None,None,None,None,None,None,None,None,0.325,0.35,0.3,None,None,None,None,None,None,None,None,None,None,None,0.325,None,None,None,None,None,0.3,None,0.3

In [45]:
print("Top ten highest accuracies")
topTen.reverse()
for top in topTen:
    print(
        f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
    )


Top ten highest accuracies
Accuracy 0.44 : subject 3, feature:dataWCV--fftData, kernelsigmoid, C:1.62
Accuracy 0.44 : subject 3, feature:-fftData, kernelsigmoid, C:1.62
Accuracy 0.44 : subject 5, feature:welchData--fftData, kernelsigmoid, C:3.88
Accuracy 0.44 : subject 5, feature:dataWCV--welchData, kernelsigmoid, C:2.75
Accuracy 0.44 : subject 5, feature:dataHRCV--fftData, kernelsigmoid, C:5.0
Accuracy 0.4 : subject 1, feature:dataWCV--dataHR, kernelsigmoid, C:3.88
Accuracy 0.4 : subject 1, feature:dataHR--welchData, kernelsigmoid, C:2.75
Accuracy 0.4 : subject 1, feature:dataHR--welchData, kernelsigmoid, C:1.62
Accuracy 0.4 : subject 1, feature:dataFFTCV--fftData, kernelrbf, C:3.88
Accuracy 0.4 : subject 1, feature:dataFFTCV--fftData, kernellinear, C:0.5


In [46]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)


print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(maxInd)
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.32602041 0.         0.33333333 0.3349359  0.34159159
 0.         0.         0.         0.        ]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395
0,0.31875,NaN,0.3,0.30625,0.30625,0.30625,0.3,0.352315,0.319907,0.30625,0.316667,0.354167,NaN,0.3,0.329167,0.31875,0.339583,0.3125,0.395833,0.395833,0.3125,0.3125,NaN,NaN,0.35,0.325,0.325,0.325,0.31875,0.340278,0.3,0.340278,0.375,NaN,0.3,NaN,0.339583,0.31875,0.3125,NaN,NaN,0.3125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.325,0.302778,0.325,0.325,0.325,NaN,NaN,0.333333,0.3125,0.34375,0.375,0.333333,0.30625,0.35,0.33125,0.325,0.3125,NaN,0.3125,0.333333,0.305556,0.305556,0.3125,NaN,0.4,0.375,0.3875,0.3125,NaN,0.3125,0.3125,0.333333,0.395833,NaN,NaN,0.325,0.35,0.3,0.354167,NaN,0.322917,0.34375,0.354167,0.354167,NaN,0.34375,0.357639,0.385417,0.368056,0.343056,NaN,NaN,NaN,NaN,NaN,0.327083,NaN,0.355556,0.327778,0.35,0.4,0.3,0.35,0.375,0.35625,0.33125,NaN,0.3,NaN,NaN,NaN,0.31875,NaN,0.3,0.341667,0.33125,0.33125,NaN,0.359259,0.352315,0.329167,0.316667,0.3125,NaN,NaN,NaN,NaN,NaN,0.3125,0.3375,0.395833,0.350694,0.385417,0.341667,NaN,0.333333,0.30625,0.3,0.3,0.3125,0.3125,0.3,NaN,NaN,0.375,NaN,0.35,0.35,0.375,0.375,0.333333,NaN,0.35,0.3,0.325,0.325,NaN,0.35,0.325,0.325,0.325,0.316667,0.3875,0.366667,0.316667,0.35,NaN,0.3,0.333333,0.3125,0.315278,0.341667,NaN,NaN,0.3,NaN,NaN,0.316667,NaN,0.3,0.316667,0.30625,0.30625,0.3125,0.354167,0.4375,0.395833,0.354167,0.333333,NaN,NaN,0.3,0.3,0.3,0.3125,0.347917,0.375,0.350694,0.395833,0.333333,NaN,0.333333,0.3125,0.333333,0.354167,NaN,NaN,0.3125,0.3125,0.354167,0.354167,NaN,NaN,0.3125,0.333333,0.333333,NaN,NaN,0.3125,0.333333,NaN,NaN,NaN,0.327083,0.354167,0.375,0.395833,NaN,NaN,NaN,0.302778,0.305556,NaN,NaN,0.35,0.35,0.35,0.35,0.3,0.354167,0.343056,0.352778,0.375,NaN,NaN,0.325,NaN,NaN,NaN,0.341667,0.316667,0.30625,0.375,0.325,0.354167,NaN,0.3,0.3,0.316667,0.316667,0.3,NaN,0.3,NaN,0.31875,NaN,NaN,0.354167,0.354167,0.354167,0.354167,0.3,NaN,0.333333,0.329167,0.372917,NaN,0.3,NaN,0.31875,0.329167,0.3125,NaN,NaN,NaN,0.3125,NaN,0.325,NaN,0.3,0.315278,0.325,0.3,NaN,NaN,0.333333,NaN,NaN,NaN,0.325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3,0.3,NaN,0.354167,NaN,NaN,0.333333,0.333333,NaN,NaN,0.3125,NaN,NaN,NaN,NaN,NaN,0.3,0.325,NaN,0.395833,NaN,NaN,0.3125,0.354167,0.375,NaN,NaN,0.3,NaN,NaN,NaN,NaN,0.309028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,NaN,0.3125,0.354167,NaN,NaN,0.35,0.325,0.325,0.354167,0.3125,0.3125,0.333333,NaN,0.333333,0.3,NaN,NaN,0.35,NaN


Max average accuracy 0.4375
195
1.0
dataWCV--welchData
sigmoid
C: 2.75


In [47]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(maxInd2)
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395
0,0.6375,0.0,0.3,0.6125,0.6125,0.6125,0.3,1.056944,0.959722,0.6125,0.633333,0.354167,0.0,0.3,0.658333,0.6375,0.679167,0.3125,0.395833,0.395833,0.3125,0.3125,0.0,0.0,0.35,0.325,0.325,0.325,0.6375,0.680556,0.3,0.680556,0.375,0.0,0.3,0.0,0.679167,0.6375,0.3125,0.0,0.0,0.3125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.325,0.605556,0.325,0.325,0.325,0.0,0.0,0.333333,0.3125,0.6875,0.75,0.333333,0.6125,0.35,0.6625,0.325,0.3125,0.0,0.3125,0.333333,0.305556,0.305556,0.3125,0.0,0.4,0.375,0.775,0.3125,0.0,0.3125,0.3125,0.333333,0.395833,0.0,0.0,0.325,0.35,0.3,0.354167,0.0,0.645833,0.6875,0.708333,0.708333,0.0,0.6875,0.715278,0.770833,0.736111,0.686111,0.0,0.0,0.0,0.0,0.0,0.654167,0.0,0.711111,0.655556,0.35,0.4,0.3,0.35,0.375,0.7125,0.6625,0.0,0.3,0.0,0.0,0.0,0.6375,0.0,0.3,0.683333,0.6625,0.6625,0.0,1.077778,1.056944,0.658333,0.633333,0.3125,0.0,0.0,0.0,0.0,0.0,0.3125,1.0125,0.395833,0.701389,0.770833,0.683333,0.0,0.333333,0.6125,0.3,0.3,0.3125,0.3125,0.3,0.0,0.0,0.375,0.0,0.35,0.35,0.375,0.375,0.333333,0.0,0.35,0.3,0.325,0.325,0.0,0.7,0.325,0.325,0.325,0.633333,0.775,0.733333,0.633333,0.35,0.0,0.3,0.333333,0.3125,0.945833,0.683333,0.0,0.0,0.3,0.0,0.0,0.633333,0.0,0.3,0.633333,0.6125,0.6125,0.3125,0.354167,0.4375,0.395833,0.354167,0.666667,0.0,0.0,0.3,0.3,0.3,0.3125,0.695833,0.375,0.701389,0.395833,0.333333,0.0,0.333333,0.3125,0.333333,0.354167,0.0,0.0,0.3125,0.3125,0.354167,0.354167,0.0,0.0,0.3125,0.333333,0.333333,0.0,0.0,0.3125,0.333333,0.0,0.0,0.0,0.654167,0.354167,0.375,0.395833,0.0,0.0,0.0,0.605556,0.305556,0.0,0.0,0.35,0.35,0.35,0.35,0.3,0.708333,0.686111,0.705556,0.375,0.0,0.0,0.325,0.0,0.0,0.0,0.683333,0.633333,0.6125,0.375,0.325,0.354167,0.0,0.3,0.3,0.633333,0.633333,0.3,0.0,0.3,0.0,0.6375,0.0,0.0,0.354167,0.354167,0.354167,0.354167,0.3,0.0,0.333333,0.658333,0.745833,0.0,0.3,0.0,0.6375,0.658333,0.3125,0.0,0.0,0.0,0.3125,0.0,0.325,0.0,0.3,0.630556,0.325,0.3,0.0,0.0,0.333333,0.0,0.0,0.0,0.325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.3,0.0,0.354167,0.0,0.0,0.333333,0.333333,0.0,0.0,0.3125,0.0,0.0,0.0,0.0,0.0,0.3,0.325,0.0,0.395833,0.0,0.0,0.3125,0.354167,0.375,0.0,0.0,0.3,0.0,0.0,0.0,0.0,0.618056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.3125,0.354167,0.0,0.0,0.35,0.325,0.325,0.354167,0.3125,0.3125,0.333333,0.0,0.333333,0.3,0.0,0.0,0.35,0.0


128
3.0
Max total significant accuracy then averaged 0.3592592592592592 
dataWCV--fftData
sigmoid
C: 1.62 


In [48]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
